In [40]:
# %pip install git+https://github.com/tensorflow/docs

import requests
import os.path
import asyncio
import json
import numpy as np
import pathlib
import pandas as pd
import seaborn as sns
import tensorflow as tf
import time
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling
from riotwatcher import TftWatcher, ApiError
from datetime import date, datetime

pd.options.mode.chained_assignment = None  # default='warn'

from utils.configuration import settings
from utils.utils import *
from tft.api import *


In [41]:
API_KEY = settings.api_key
ASSETS_DIR = settings.assets_dir
SERVER = 'na1'  # euw1 na1 kr oc1
LEAGUE='challengers' # challengers grandmasters
MAX_COUNT = settings.max_count
LATEST_RELEASE = '12.12.450.4196' # '12.12.450.4196' Version 12.12.448.6653 12.11.446.9344
PATCH = date(2022, 7, 1)

TARGETNAME = 'placement'

In [42]:
summoners_df = pd.read_pickle(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_summoners.pickle'))

In [43]:
summoners_df

,id,accountId,puuid,name,profileIconId,revisionDate,summonerLevel,summonerId,summonerName,leaguePoints,rank,wins,losses,veteran,inactive,freshBlood,hotStreak
0,mnqh3k_YRehdzyI4g1MenDTSPsTloRdrdv_uNmoGbJBHfVYp,WWx1zaPphzE7nwbLLkiyX-49rlRIMBdBe9iMm0kX_1_ppA...,ZZoVhG5NMRULeewjhAwEBZ3u_5KQY38CjFPKBb5UEpxgZ_...,ItzYaBoyBeasty,1477,1657030102000,183,mnqh3k_YRehdzyI4g1MenDTSPsTloRdrdv_uNmoGbJBHfVYp,ItzYaBoyBeasty,484,I,187,158,False,False,True,False
1,JBLh5joNuzgo-mok7-0tdt7woIeIjbQRcad_2BOFVnOWkVo,ICaGttJK02As_WhaByY6hrvOPQlASVzXJ52SLfbHGVybWW0,dulwPbGI9DD0dqfsdvSwBvJin59e4ZJesOwgfpZ6i8zRmK...,Rekk,7,1657001097000,169,JBLh5joNuzgo-mok7-0tdt7woIeIjbQRcad_2BOFVnOWkVo,Rekk,541,I,190,147,False,False,True,False
2,F076SSJmWsKsPULh8XD2yr_CcrGzmberIL9oH6WiQ4sydzT6,5CXmlT14PxMEPUOluZZ6VnG0iyHHkcLFtbhFSdpDzWUNWQ...,miECR0OOaJlxGXdFIC3_icU8aPZ9SCTmzrQuuMkLvo64Zx...,Dıshsoap,4368,1657029659000,17,F076SSJmWsKsPULh8XD2yr_CcrGzmberIL9oH6WiQ4sydzT6,Dıshsoap,1322,I,227,102,True,False,False,False
3,uf6ZqEaHhg9VYQxjiLpGJgUwOBB6X89Hnrx88QVbfcW4ovEK,yhcCov8r0-BCri_s7u9Q5gWMW__mGGByo9x54QR6E7sG_H...,XPEnythbslhe7S-0HfEw7CuMFlTWra83g9aivkbQvisG2t...,iCopyKeane,5169,1656979779000,19,uf6ZqEaHhg9VYQxjiLpGJgUwOBB6X89Hnrx88QVbfcW4ovEK,iCopyKeane,550,I,175,146,False,False,True,False
4,K5pJ4JIlqzeSs4vqW2geDqfaPYMkDj-I3oO5GKFSGVAyZfc,yoZaGYEip0GZTryRIcFICdqPF7Ss2NE8naspIB-iyZrWmw,fb5jK6O_ukbKVChTVy72j0BwKOf6j7mTfoejrErTrsLTQM...,Liquid Robin,4692,1657017135000,118,K5pJ4JIlqzeSs4vqW2geDqfaPYMkDj-I3oO5GKFSGVAyZfc,Liquid Robin,1060,I,136,70,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,dOMGV91pp2e7mqHOffMIKZn9lNMpxljwpHtDrHuiKrnF6OYl,ZvgfFlXsNebfxn9dUheDa0H4Dhb-4vVCzSRAkv2NhM28w7...,tiwoB5V6ZKlhgzdc6j2mLIfou3-tf9BopY-5y7acb0ZKdb...,WizardTFT,5369,1657003931000,62,dOMGV91pp2e7mqHOffMIKZn9lNMpxljwpHtDrHuiKrnF6OYl,WizardTFT,546,I,149,103,False,False,True,False
160,iK7HHHIkQARvIiI98bNXpcmjRSyB1u4-KLuxPhbjHLPL5gc,_Y-IPIRIDt9C4KH7CpeJ4esQ-KssQ3Ongm2EjhEq8140hA,RfFChF-FG1121tg4frHGwr5UIeQWbjNwatqcVaVypMicfO...,QualityHooker,4641,1656950273000,224,iK7HHHIkQARvIiI98bNXpcmjRSyB1u4-KLuxPhbjHLPL5gc,QualityHooker,548,I,135,104,False,False,True,False
161,JnwguzzSoBb04ADe3G0cZ7eqsla2IRd1t8FZ5nkMlBlgTqU,QGfImgWzzX9pBECd2ilPH4lSnfsR4C8w86vadIMggOgQRA,hF0wOAokBIAA2XBx8nAmSCLkZVvNi9pAJ1JS_fuBjk61qT...,theOnLy1z,588,1656783709000,137,JnwguzzSoBb04ADe3G0cZ7eqsla2IRd1t8FZ5nkMlBlgTqU,theOnLy1z,827,I,153,106,False,False,False,False
162,Br5jVXksU0hNSDBSfzmoCQYSsCW91VDgEclB66NMkJETJlM,X_zpdzBWpuAxdcPC5-j_TVu7kUaRPoBVGnpYcbek5JU_jO4,3uTHsT1uOzOaXoAbzlfu84s1CuKOa8QFCLmIuERsxzN-S-...,SuperF1sh,3151,1656902936000,96,Br5jVXksU0hNSDBSfzmoCQYSsCW91VDgEclB66NMkJETJlM,SuperF1sh,533,I,181,156,False,False,True,True


In [44]:

# Get all unique matches_id from assets dir
matches_asset = load_matches(summoners_df, server=SERVER)
matches_id = [match['metadata']['match_id'] for match in matches_asset]
seen = set()
uniq_matches_id = [
    x for x in matches_id if x not in seen and not seen.add(x)]

In [45]:
len(uniq_matches_id)

5291

In [46]:
len(matches_asset)

8796

In [47]:
seen = set()
seen_add = seen.add
uniq_matches = [x for x in matches_asset if x['metadata']['match_id'] not in seen and not seen_add(x['metadata']['match_id'])]

In [48]:
len(uniq_matches)

5291

In [49]:
matches_asset[300]['info']['game_version']

'Version 12.12.448.6653 (Jun 16 2022/17:54:50) [PUBLIC] <Releases/12.12>'

In [50]:
latest_matches = [match for match in uniq_matches if (
    LATEST_RELEASE in match['info']['game_version'])]


In [51]:
latest_patch_matches = [match for match in uniq_matches if (LATEST_RELEASE in match['info']['game_version'])
                        and (PATCH <= date.fromtimestamp(match['info']['game_datetime']/1000.0))]


In [52]:
len(latest_matches)

3497

In [53]:
len(latest_patch_matches)

1597

In [54]:
# latest_matches[0]['info']

In [242]:
from typing import List


def process_matches(df) -> List:
    matches_array = []

    for match_row in df:
        match_id = match_row['metadata']['match_id']

        for participant in match_row['info']['participants']:
            match = {}
            match['match_id'] = match_id
            # match['level'] = participant['level']
            match['placement'] = participant['placement']
            # match['players_eliminated'] = participant['players_eliminated']
            # match['total_damage_to_players'] = participant['total_damage_to_players']

            for augment_index, augment in enumerate(participant['augments']):
                # if augment == 'TFT7_Augment_GuildLootHR':
                #     augment = 'TFT7_Augment_BandOfThieves1'
                match[f'augment{augment_index}'] = augment

            for trait_index, trait in enumerate(participant['traits']):
                match[f'{trait["name"]}'] = trait["tier_current"]

            for unit_index, unit in enumerate(participant['units']):
                match[f'{unit["character_id"]}'] = unit["tier"]
                match['TFT7_TrainerDragon_item1'] = 'None'
                match['TFT7_TrainerDragon_item2'] = 'None'
                for item_index, item in enumerate(unit['itemNames']):
                    match[f'{unit["character_id"]}_item{item_index}'] = item.split('_')[-1]

            matches_array.append(match)

    return matches_array


In [243]:
matches_array = process_matches(latest_matches)
matches_patch_array = process_matches(latest_patch_matches)

In [244]:
matches_array[0]
# len(matches_array)

{'match_id': 'NA1_4363861621',
 'placement': 3,
 'augment0': 'TFT6_Augment_TargetDummies',
 'augment1': 'TFT6_Augment_Meditation2',
 'augment2': 'TFT6_Augment_CelestialBlessing2',
 'Set7_Assassin': 1,
 'Set7_Bruiser': 2,
 'Set7_Dragon': 1,
 'Set7_Guardian': 0,
 'Set7_Legend': 0,
 'Set7_Mage': 0,
 'Set7_Ragewing': 0,
 'Set7_Tempest': 1,
 'Set7_Warrior': 0,
 'Set7_Whispers': 3,
 'TFT7_Thresh': 1,
 'TFT7_TrainerDragon_item1': 'None',
 'TFT7_TrainerDragon_item2': 'None',
 'TFT7_Qiyana': 2,
 'TFT7_Shen': 2,
 'TFT7_Sylas': 2,
 'TFT7_DragonPurple': 2,
 'TFT7_DragonPurple_item0': 'UnstableConcoction',
 'TFT7_DragonPurple_item1': 'GuinsoosRageblade',
 'TFT7_DragonPurple_item2': 'MadredsBloodrazor',
 'TFT7_Ornn': 2,
 'TFT7_Ornn_item0': 'DragonsClaw',
 'TFT7_Ornn_item1': 'NegatronCloak',
 'TFT7_Ornn_item2': 'Redemption',
 'TFT7_Pyke': 1,
 'TFT7_Pyke_item0': 'InfinityEdge',
 'TFT7_Pyke_item1': 'GuardianAngel',
 'TFT7_Pyke_item2': 'HextechGunblade'}

In [245]:
matches_league_df = pd.json_normalize(matches_array)
matches_league_patch_df = pd.json_normalize(matches_patch_array)

In [246]:
matches_league_df

,match_id,placement,augment0,augment1,augment2,Set7_Assassin,Set7_Bruiser,Set7_Dragon,Set7_Guardian,Set7_Legend,...,TFT7_Sett_item0,TFT7_Senna,TFT7_Senna_item0,TFT7_Heimerdinger_item2,TFT7_Sett_item1,TFT7_Lillia_item2,TFT7_Thresh_item2,TFT7_Sett_item2,TFT7_Senna_item1,TFT7_Senna_item2
0,NA1_4363861621,3,TFT6_Augment_TargetDummies,TFT6_Augment_Meditation2,TFT6_Augment_CelestialBlessing2,1.0,2.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NA1_4363861621,7,TFT6_Augment_HyperRoll,TFT6_Augment_CelestialBlessing2,TFT6_Augment_FuturePeepers,1.0,1.0,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NA1_4363861621,2,TFT7_Augment_GuildEmblem,TFT7_Augment_LastStand,TFT6_Augment_Ascension,1.0,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NA1_4363861621,6,TFT6_Augment_TriForce2,TFT7_Augment_BruiserPersonalTraining,TFT7_Augment_SwiftshotEmblem,NaN,2.0,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NA1_4363861621,1,TFT6_Augment_SalvageBin,TFT6_Augment_FuturePeepers,TFT6_Augment_CyberneticUplink2,NaN,NaN,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27971,NA1_4355960440,2,TFT7_Augment_RevelEmblem2,TFT7_Augment_RevelTrait,TFT6_Augment_SecondWind2,0.0,1.0,1.0,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27972,NA1_4355960440,4,TFT6_Augment_CyberneticImplants3,TFT6_Augment_Diversify1,TFT6_Augment_TomeOfTraits1,NaN,NaN,1.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27973,NA1_4355960440,1,TFT6_Augment_Electrocharge3,TFT6_Augment_Recombobulator,TFT6_Augment_SalvageBin,0.0,NaN,1.0,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27974,NA1_4355960440,7,TFT6_Augment_CelestialBlessing3,TFT7_Augment_BestFriends1,TFT7_Augment_SwiftshotEmblem,1.0,1.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [247]:
matches_league_df[matches_league_df.isnull().T.any()]

,match_id,placement,augment0,augment1,augment2,Set7_Assassin,Set7_Bruiser,Set7_Dragon,Set7_Guardian,Set7_Legend,...,TFT7_Sett_item0,TFT7_Senna,TFT7_Senna_item0,TFT7_Heimerdinger_item2,TFT7_Sett_item1,TFT7_Lillia_item2,TFT7_Thresh_item2,TFT7_Sett_item2,TFT7_Senna_item1,TFT7_Senna_item2
0,NA1_4363861621,3,TFT6_Augment_TargetDummies,TFT6_Augment_Meditation2,TFT6_Augment_CelestialBlessing2,1.0,2.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NA1_4363861621,7,TFT6_Augment_HyperRoll,TFT6_Augment_CelestialBlessing2,TFT6_Augment_FuturePeepers,1.0,1.0,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NA1_4363861621,2,TFT7_Augment_GuildEmblem,TFT7_Augment_LastStand,TFT6_Augment_Ascension,1.0,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NA1_4363861621,6,TFT6_Augment_TriForce2,TFT7_Augment_BruiserPersonalTraining,TFT7_Augment_SwiftshotEmblem,NaN,2.0,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NA1_4363861621,1,TFT6_Augment_SalvageBin,TFT6_Augment_FuturePeepers,TFT6_Augment_CyberneticUplink2,NaN,NaN,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27971,NA1_4355960440,2,TFT7_Augment_RevelEmblem2,TFT7_Augment_RevelTrait,TFT6_Augment_SecondWind2,0.0,1.0,1.0,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27972,NA1_4355960440,4,TFT6_Augment_CyberneticImplants3,TFT6_Augment_Diversify1,TFT6_Augment_TomeOfTraits1,NaN,NaN,1.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27973,NA1_4355960440,1,TFT6_Augment_Electrocharge3,TFT6_Augment_Recombobulator,TFT6_Augment_SalvageBin,0.0,NaN,1.0,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27974,NA1_4355960440,7,TFT6_Augment_CelestialBlessing3,TFT7_Augment_BestFriends1,TFT7_Augment_SwiftshotEmblem,1.0,1.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Sort and reorder columns

In [248]:
def reorder_df_col(df):
    fixed_cols = ['placement', 'match_id', 'augment0', 'augment1', 'augment2']
    all_cols = df.columns
    to_sort_cols = list(set(all_cols) - set(fixed_cols))

    return df.reindex(columns=fixed_cols + sorted(to_sort_cols))


In [249]:
matches_league_df = reorder_df_col(matches_league_df)
matches_league_patch_df = reorder_df_col(matches_league_patch_df)

# Output dataframes

In [250]:
matches_league_df.to_pickle(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_matches.pickle'))
matches_league_df.to_csv(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_matches.csv'), index=False)
matches_league_patch_df.to_pickle(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_{PATCH}_matches.pickle'))
matches_league_patch_df.to_csv(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_{PATCH}_matches.csv'), index=False)

# Preprocessing

In [251]:
raw_df = matches_league_df.copy()
def impute(df):
    for name in df.select_dtypes("number"):
        df[name] = df[name].fillna(0)
    for name in df.select_dtypes("object"):
        df[name] = df[name].fillna("None")
    return df
raw_df = impute(raw_df)

In [252]:
match_id_df = raw_df['match_id']
X = raw_df.drop(['match_id'], axis=1)
y = X.pop(TARGETNAME)
X.fillna('', inplace=True)
numeric_cols = X.select_dtypes(include=np.number).columns.tolist()
categorical_cols = X.select_dtypes(include=['object','category']).columns.tolist()
traits_col = [s for s in numeric_cols if "Set7" in s]
units_col = [s for s in numeric_cols if "TFT7" in s]
augments_col = ['augment0', 'augment1', 'augment2']
items_col = [s for s in categorical_cols if s not in augments_col]
df_unique = X.nunique().to_frame().reset_index()
df_unique.columns = ['Variable','DistinctCount']
unique_items_set = {y for col in items_col for y in X[col].unique().tolist()}
unique_augments_set = {y for col in augments_col for y in X[col].unique().tolist()}
X[f'items_count'] = X[items_col].apply(lambda row: sum(x != 'None' for x in row), axis=1)
X[f'traits_sum'] = X[traits_col].sum(axis=1)
X[f'units_sum'] = X[units_col].sum(axis=1)
X.iloc[X[f'units_sum'].idxmax()]

augment0          TFT7_Augment_MageConference
augment1              TFT6_Augment_ThriftShop
augment2             TFT6_Augment_SecondWind1
Set7_Assassin                             0.0
Set7_Astral                               1.0
                             ...             
TFT7_Zoe_item1                           None
TFT7_Zoe_item2                           None
items_count                                12
traits_sum                                9.0
units_sum                                30.0
Name: 7625, Length: 270, dtype: object

In [253]:
numeric_cols = X.select_dtypes(include=np.number).columns.tolist()
categorical_cols = X.select_dtypes(include=['object','category']).columns.tolist()
X[numeric_cols] = X[numeric_cols].applymap(np.int64)
matches_df = X.copy()
matches_df[TARGETNAME] = y

In [254]:
def get_unit_items_ranking(df: matches_df, unit: str):
    # filter and melt the dataframe
    df = df.filter(regex=f'placement|{unit}_item0|{unit}_item1|{unit}_item2')
    df[f'unit'] = f'{unit}' # fill in current unit
    # join 3 items to 1 column
    df[f'{unit}_items'] = df[[f'{unit}_item0', f'{unit}_item1', f'{unit}_item2']].apply(lambda row: ', '.join(row.values.astype(str)), axis=1)
    # sort items for unique combination
    df[f'{unit}_items'] = df[f'{unit}_items'].apply(lambda x: ', '.join(sorted(x.split(', '))))
    m = df.melt(
        ['placement',f'unit'], value_name=f'{unit}_items_grp') #, value_vars=[f'{unit}_items', f'{unit}']
    # group and aggregate mean/median average_placement
    dct = {'value_count': (f'{unit}_items_grp', 'count'),
           'average_placement': ('placement', 'mean')}
    return m.groupby([f'unit', f'{unit}_items_grp'], as_index=False).agg(**dct).sort_values(by='average_placement')

In [255]:
# Get top5 
top5_list = []
for unit in units_col:
    df = get_unit_items_ranking(df = matches_df, unit=unit)
    df = df[df['value_count']>=12][:5] #Top 5 with counts >= 12
    top5_list.extend(df.values)

In [256]:
top5_df = pd.DataFrame(top5_list, columns=['unit', 'items',	'value_count',	'average_placement'])

In [257]:
top5_df.to_csv(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_{PATCH}_top5_items.csv'), index=False)

In [258]:
pd.options.display.max_colwidth = 100

In [259]:
top5_df.groupby('unit').head(1)

,unit,items,value_count,average_placement
0,TFT7_Aatrox,TitanicHydra,22,3.045455
5,TFT7_Anivia,"ArchangelsStaff, JeweledGauntlet, StatikkShiv",16,2.875000
10,TFT7_AoShin,"ArchangelsStaff, Quicksilver, SpearOfShojin",17,1.941176
15,TFT7_Ashe,"Deathblade, None, None",15,3.133333
20,TFT7_AurelionSol,"ArchangelsStaff, HextechGunblade, MageEmblemItem",15,4.200000
25,TFT7_Bard,DragonsClaw,13,2.307692
30,TFT7_Braum,BruiserEmblemItem,18,3.333333
35,TFT7_Corki,"GuinsoosRageblade, InfinityEdge, StatikkShiv",14,2.785714
40,TFT7_Diana,LastWhisper,16,3.062500
45,TFT7_DragonBlue,"GuinsoosRageblade, Quicksilver, RevelEmblemItem",40,3.075000
